In [ ]:
import requests
import os
from tqdm.auto import tqdm

## Пример скачивания 1 файла

In [ ]:
# Full path - пример
# https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet

os.makedirs("nyc_taxi_data", exist_ok=True) # Создаем папку

base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data" # базовый путь к файлу на сервере
filename = f"yellow_tripdata_2024-01.parquet"                # имя файла
url = f"{base_url}/{filename}"                               # Полный URL
filepath = os.path.join("nyc_taxi_data", filename)  # Путь на диске (корректное объединение пути с помощью os.path.join())

In [ ]:
# весь блок одеваем в try except, 
# чтобы при случае отсутствия соединения или файла программа не крашнулась
try:
    response = requests.get(url, stream=True)  # 1. Отправляем HTTP GET запрос к URL для скачивания файла
                                        #     stream=True - КРИТИЧЕСКИ важный параметр:
                                        # Без stream=True: requests загружает ВЕСЬ файл в оперативную память сразу
                                        #   С stream=True: файл загружается ЧАСТЯМИ (чанками)
    
    response.raise_for_status() # 2. ПРОВЕРКА НА ОШИБКИ HTTP: автоматически проверяет код ответа сервера
                                # - Если код 200 (OK) - ничего не происходит, продолжаем выполнение
                                # - Если код 4xx (клиентская ошибка) или 5xx (серверная ошибка) - 
                                #   ВЫБРАСЫВАЕТСЯ ИСКЛЮЧЕНИЕ requests.exceptions.HTTPError
                                
    with open(filepath, 'wb') as f: #  ОТКРЫВАЕМ ФАЙЛ ДЛЯ ЗАПИСИ в БИНАРНОМ РЕЖИМЕ:
                                    #  'wb' означает:
                                    #    w - write (запись)
                                    #    b - binary (бинарный режим - для файлов, которые не являются текстом)
                                    #  Конструкция 'with' гарантирует, что файл будет КОРРЕКТНО ЗАКРЫТ 
                                    #  даже если возникнет ошибка во время записи
        for chunk in response.iter_content(chunk_size=8192):    # ПОСТРОЧНОЕ ЧТЕНИЕ И ЗАПИСА ФАЙЛА:
                                                                # response.iter_content() - это ГЕНЕРАТОР, который возвращает файл 
                                                                # небольшими частями (чанками) вместо одной большой порции
                                                                # chunk_size=8192 - размер одного кусочка в байтах (8 КБ)
            if chunk:  # 5. ФИЛЬТРАЦИЯ ПУСТЫХ ЧАНКОВ:
                       # Иногда iter_content может возвращать пустые чанки (None или b'')
                       # Эта проверка гарантирует, что мы записываем ТОЛЬКО данные
                       # и не выполняем лишних операций записи
                f.write(chunk)  # 6. ЗАПИСЬ ЧАНКА НА ДИСК:
                                # Каждый кусочек данных (8 КБ) записывается в файл
                                # Это происходит МНОГОКРАТНО, пока весь файл не будет скачан
    
    file_size = os.path.getsize(filepath) / (1024 * 1024) # os.path.getsize(filepath) - получает размер файла в БАЙТАХ
    result = f"✓ {filename} ({file_size:.1f} MB)"
    
except Exception as e:
    result = f"✗ {filename}: {e}"

print(f'Результат: {result}')

## Скачиваем пакетом

In [ ]:
def download_simple():
    """Упрощенная загрузка файлов по очереди"""
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data"
    os.makedirs("nyc_taxi_data", exist_ok=True)
    
    results = []
    
    for month in tqdm(range(1, 13), desc="Скачивание файла"):
        filename = f"yellow_tripdata_2024-{month:02d}.parquet"
        url = f"{base_url}/{filename}"
        filepath = os.path.join("nyc_taxi_data", filename)
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            file_size = os.path.getsize(filepath) / (1024 * 1024)
            results.append(f"✓ {filename} ({file_size:.1f} MB)")
            
        except Exception as e:
            results.append(f"✗ {filename}: {e}")
    
    return results

print("🚕 Загрузка NYC Yellow Taxi 2024 (упрощенная версия)...")
results = download_simple()

print("\nРезультаты:")
for result in results:
    print(result)

### Скачиваем сразу в MinIO

### Маленькие файлы, грузятся в оперативку

In [ ]:
from minio import Minio
from minio.error import S3Error
import requests
from tqdm import tqdm
import io

def download_to_minio_optimized():
    """Оптимизированная загрузка в MinIO"""
    # Настройка клиента MinIO
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin",
        secret_key="minioadmin",
        secure=False
    )

    bucket_name = "bronze"
    prefix = "nyc-taxi-data"
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data"
    
    # Создаем бакет если нужно
    try:
        if not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"✓ Бакет {bucket_name} создан")
    except S3Error as e:
        return [f"✗ Ошибка бакета: {e}"]
    
    results = []
    
    for month in tqdm(range(1, 2), desc="Загрузка в MinIO"):
        filename = f"yellow_tripdata_2024-{month:02d}.parquet"
        url = f"{base_url}/{filename}"
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Получаем размер файла
            file_size = int(response.headers.get('content-length', 0))
            
            # Потоковая загрузка в MinIO
            file_stream = io.BytesIO()
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file_stream.write(chunk)
            
            file_stream.seek(0)  # Возвращаемся в начало потока
            
            minio_client.put_object(
                bucket_name=bucket_name,
                object_name=f"{prefix}/{filename}",
                data=file_stream,
                length=file_size
            )
            
            results.append(f"✓ s3a://{prefix}/{filename} ({file_size / (1024 * 1024):.1f} MB)")
            
        except Exception as e:
            results.append(f"✗ {filename}: {e}")
    
    return results

# Использование
print("🚕 Загрузка в MinIO...")
results = download_to_minio_optimized()

print("\nРезультаты:")
for result in results:
    print(result)

### Для больших файлов

In [ ]:
from minio import Minio
from minio.error import S3Error
import requests
from tqdm import tqdm
import os
import tempfile

def download_to_minio_optimized():
    """Оптимизированная загрузка в MinIO для больших файлов"""
    # Настройка клиента MinIO
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin",
        secret_key="minioadmin",
        secure=False
    )

    bucket_name = "bronze"
    prefix = "nyc-taxi-data"
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data"
    
    # Создаем бакет если нужно
    try:
        if not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"✓ Бакет {bucket_name} создан")
    except S3Error as e:
        return [f"✗ Ошибка бакета: {e}"]
    
    results = []
    
    for month in tqdm(range(1, 2), desc="Загрузка в MinIO"):
        filename = f"yellow_tripdata_2024-{month:02d}.parquet"
        url = f"{base_url}/{filename}"
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Создаем временный файл вместо использования памяти
            with tempfile.NamedTemporaryFile(delete=False, suffix='.parquet') as temp_file:
                temp_path = temp_file.name
                print(f"📁 Временный файл: {temp_path}")  # ← ДОБАВЬ ЭТУ СТРОКУ
                
                # Скачиваем файл на диск порциями
                total_size = int(response.headers.get('content-length', 0))
                downloaded = 0
                
                for chunk in response.iter_content(chunk_size=8192 * 8):  # Увеличили chunk_size
                    if chunk:
                        temp_file.write(chunk)
                        downloaded += len(chunk)
            
            # Получаем реальный размер файла
            file_size = os.path.getsize(temp_path)
            
            # Загружаем в MinIO с диска (не из памяти)
            minio_client.fput_object(  # ← ИЗМЕНЕНО: используем fput_object вместо put_object
                bucket_name=bucket_name,
                object_name=f"{prefix}/{filename}",
                file_path=temp_path  # ← ИЗМЕНЕНО: загрузка с диска
            )
            
            # Удаляем временный файл
            os.unlink(temp_path)
            
            results.append(f"✓ s3a://{bucket_name}/{prefix}/{filename} ({file_size / (1024 * 1024):.1f} MB)")
            
        except Exception as e:
            # Удаляем временный файл в случае ошибки
            if 'temp_path' in locals():
                try:
                    os.unlink(temp_path)
                except:
                    pass
            results.append(f"✗ {filename}: {e}")
    
    return results

In [ ]:
# Использование
print("🚕 Загрузка в MinIO...")
results = download_to_minio_optimized()

print("\nРезультаты:")
for result in results:
    print(result)

# С проверкой файлов на сайте

In [45]:
from minio import Minio
from minio.error import S3Error
import requests
from tqdm import tqdm
import os
import tempfile

def get_available_remote_files(base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                               filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                               year=2024):
    """Проверить какие файлы фактически существуют на сайте"""
    available_files = []
    
    print("🔍 Проверка доступных файлов на сайте...")
    
    for month in tqdm(range(1, 13), desc="Проверка месяца"):
        filename = filename_template.format(year=year, month=month)
        url = f"{base_url}/{filename}"
        
        try:
            # Делаем HEAD запрос чтобы проверить существование файла
            response = requests.head(url, timeout=10)
            if response.status_code == 200:
                available_files.append(filename)
                print(f"  ✓ {filename} - доступен")
            else:
                print(f"  ✗ {filename} - недоступен (код: {response.status_code})")
                
        except requests.exceptions.RequestException as e:
            print(f"  ✗ {filename} - ошибка: {e}")
    
    return available_files

def get_local_minio_files(minio_client, 
                          bucket_name="bronze", 
                          prefix="nyc-taxi-data"):
    """Получить список файлов в MinIO"""
    local_files = []
    try:
        objects = minio_client.list_objects(bucket_name, prefix=prefix, recursive=True)
        for obj in objects:
            # Извлекаем имя файла из полного пути
            filename = obj.object_name.replace(f"{prefix}/", "")
            local_files.append(filename)
    except S3Error as e:
        print(f"Ошибка при чтении бакета: {e}")
    
    return local_files

def download_missing_files(bucket_name="bronze", 
                           prefix="nyc-taxi-data",
                           base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                           filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                           year=2024):
    """Загрузка только отсутствующих файлов в MinIO"""
    # Настройка клиента MinIO
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin",
        secret_key="minioadmin",
        secure=False
    )
    
    # Создаем бакет если нужно
    try:
        if not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"✓ Бакет {bucket_name} создан")
    except S3Error as e:
        return [f"✗ Ошибка бакета: {e}"]
    
    # Получаем списки файлов
    remote_files = get_available_remote_files(base_url, filename_template, year)
    local_files = get_local_minio_files(minio_client, bucket_name, prefix)
    
    # Находим отсутствующие файлы
    missing_files = list(set(remote_files) - set(local_files))

    # Блок статистики
    print(f"\n📊 СТАТИСТИКА:")

    print(f"• Загружено в MinIO: {len(local_files)} файл(ов)")    

    print()    
    print(f"• Доступно на сайте: {len(remote_files)} файл(ов)")
    for file in sorted(remote_files):
        print(f"     - {file}")

    print()
    if missing_files:
        print(f"• Из них отсутствует в MinIO: {len(missing_files)} файл(ов)")
        for file in sorted(missing_files):
            print(f"     - {file}")
    
    if not missing_files:
        return ["✅ Все доступные файлы уже загружены"]
    
    results = []
    
    # Скачиваем только отсутствующие файлы
    for filename in tqdm(missing_files, desc="Загрузка недостающих"):
        url = f"{base_url}/{filename}"
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Создаем временный файл
            with tempfile.NamedTemporaryFile(delete=False, suffix='.parquet') as temp_file:
                temp_path = temp_file.name
                
                # Скачиваем файл на диск
                total_size = int(response.headers.get('content-length', 0))
                for chunk in response.iter_content(chunk_size=8192 * 8):
                    if chunk:
                        temp_file.write(chunk)
            
            # Получаем реальный размер файла
            file_size = os.path.getsize(temp_path)
            
            # Загружаем в MinIO
            minio_client.fput_object(
                bucket_name=bucket_name,
                object_name=f"{prefix}/{filename}",
                file_path=temp_path
            )
            
            # Удаляем временный файл
            os.unlink(temp_path)
            
            results.append(f"✓ {filename} ({file_size / (1024 * 1024):.1f} MB)")
            
        except Exception as e:
            # Удаляем временный файл в случае ошибки
            if 'temp_path' in locals():
                try:
                    os.unlink(temp_path)
                except:
                    pass
            results.append(f"✗ {filename}: {e}")
    
    return results

def show_current_state(bucket_name="bronze", 
                       prefix="nyc-taxi-data",
                       base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                       filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                       year=2024):
    """Показать текущее состояние файлов"""
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin", 
        secret_key="minioadmin",
        secure=False
    )
    
    remote_files = get_available_remote_files(base_url, filename_template, year)
    local_files = get_local_minio_files(minio_client, bucket_name, prefix)
    missing_files = list(set(remote_files) - set(local_files))
    
    print("\n📊 ТЕКУЩЕЕ СОСТОЯНИЕ:")
    print(f"• Загружено в MinIO: {len(local_files)} файл(ов)")

    print()
    print(f"• Доступно на сайте: {len(remote_files)} файл(ов)")
    for file in sorted(remote_files):
        print(f"     - {file}")    

    print()
    print(f"• Из них отсутствует в MinIO: {len(missing_files)} файл(ов)")
    if missing_files:
        for file in sorted(missing_files):
            print(f"     - {file}")

In [46]:
# Передаем параметры источника и получателя
results = download_missing_files(
    bucket_name="bronze", 
    prefix="nyc-taxi-data",
    base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
    filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
    year=2022
)

print("Результаты дозагрузки:")
for result in sorted(results):
    print(result)

🔍 Проверка доступных файлов на сайте...


Проверка месяца:   8%|▊         | 1/12 [00:00<00:01,  7.07it/s]

  ✓ yellow_tripdata_2022-01.parquet - доступен


Проверка месяца:  17%|█▋        | 2/12 [00:00<00:01,  7.15it/s]

  ✓ yellow_tripdata_2022-02.parquet - доступен


...

  ✓ yellow_tripdata_2022-11.parquet - доступен


Проверка месяца: 100%|██████████| 12/12 [00:01<00:00,  6.79it/s]


  ✓ yellow_tripdata_2022-12.parquet - доступен

📊 СТАТИСТИКА:
• Загружено в MinIO: 33 файл(ов)

• Доступно на сайте: 12 файл(ов)
     - yellow_tripdata_2022-01.parquet
     - yellow_tripdata_2022-02.parquet
     - yellow_tripdata_2022-03.parquet
     - yellow_tripdata_2022-04.parquet
     - yellow_tripdata_2022-05.parquet
     - yellow_tripdata_2022-06.parquet
     - yellow_tripdata_2022-07.parquet
     - yellow_tripdata_2022-08.parquet
     - yellow_tripdata_2022-09.parquet
     - yellow_tripdata_2022-10.parquet
     - yellow_tripdata_2022-11.parquet
     - yellow_tripdata_2022-12.parquet

• Из них отсутствует в MinIO: 12 файл(ов)
     - yellow_tripdata_2022-01.parquet
     - yellow_tripdata_2022-02.parquet
     - yellow_tripdata_2022-03.parquet
     - yellow_tripdata_2022-04.parquet
     - yellow_tripdata_2022-05.parquet
     - yellow_tripdata_2022-06.parquet
     - yellow_tripdata_2022-07.parquet
     - yellow_tripdata_2022-08.parquet
     - yellow_tripdata_2022-09.parquet
     - ye

Загрузка недостающих: 100%|██████████| 12/12 [00:50<00:00,  4.19s/it]

Результаты дозагрузки:
✓ yellow_tripdata_2022-01.parquet (36.4 MB)
✓ yellow_tripdata_2022-02.parquet (43.5 MB)
✓ yellow_tripdata_2022-03.parquet (53.1 MB)
✓ yellow_tripdata_2022-04.parquet (52.7 MB)
✓ yellow_tripdata_2022-05.parquet (53.0 MB)
✓ yellow_tripdata_2022-06.parquet (52.8 MB)
✓ yellow_tripdata_2022-07.parquet (47.1 MB)
✓ yellow_tripdata_2022-08.parquet (47.4 MB)
✓ yellow_tripdata_2022-09.parquet (47.3 MB)
✓ yellow_tripdata_2022-10.parquet (54.4 MB)
✓ yellow_tripdata_2022-11.parquet (47.8 MB)
✓ yellow_tripdata_2022-12.parquet (51.2 MB)


In [44]:
# Показываем состояние  
show_current_state(bucket_name="bronze", 
                   prefix="nyc-taxi-data",
                   base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                   filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                   year=2022)

🔍 Проверка доступных файлов на сайте...


Проверка месяца:   8%|▊         | 1/12 [00:00<00:05,  2.03it/s]

  ✓ yellow_tripdata_2022-01.parquet - доступен


Проверка месяца:  17%|█▋        | 2/12 [00:00<00:04,  2.09it/s]

  ✓ yellow_tripdata_2022-02.parquet - доступен


...

  ✓ yellow_tripdata_2022-08.parquet - доступен


Проверка месяца:  83%|████████▎ | 10/12 [00:04<00:00,  2.93it/s]

  ✓ yellow_tripdata_2022-09.parquet - доступен
  ✓ yellow_tripdata_2022-10.parquet - доступен


Проверка месяца: 100%|██████████| 12/12 [00:04<00:00,  2.73it/s]

  ✓ yellow_tripdata_2022-11.parquet - доступен
  ✓ yellow_tripdata_2022-12.parquet - доступен

📊 ТЕКУЩЕЕ СОСТОЯНИЕ:
• Загружено в MinIO: 33 файл(ов)

• Доступно на сайте: 12 файл(ов)
     - yellow_tripdata_2022-01.parquet
     - yellow_tripdata_2022-02.parquet
     - yellow_tripdata_2022-03.parquet
     - yellow_tripdata_2022-04.parquet
     - yellow_tripdata_2022-05.parquet
     - yellow_tripdata_2022-06.parquet
     - yellow_tripdata_2022-07.parquet
     - yellow_tripdata_2022-08.parquet
     - yellow_tripdata_2022-09.parquet
     - yellow_tripdata_2022-10.parquet
     - yellow_tripdata_2022-11.parquet
     - yellow_tripdata_2022-12.parquet

• Из них отсутствует в MinIO: 12 файл(ов)
  - yellow_tripdata_2022-01.parquet
  - yellow_tripdata_2022-02.parquet
  - yellow_tripdata_2022-03.parquet
  - yellow_tripdata_2022-04.parquet
  - yellow_tripdata_2022-05.parquet
  - yellow_tripdata_2022-06.parquet
  - yellow_tripdata_2022-07.parquet
  - yellow_tripdata_2022-08.parquet
  - yellow_tripdat